In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
import cv2
import os
import re
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import models
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import shutil
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
np.set_printoptions(linewidth=10000)
import imageio
from datetime import datetime as dt
from PIL import Image
import cv2
from keras.models import load_model
from cv2 import resize,INTER_AREA
from sklearn.metrics import roc_curve,auc

In [ ]:
load = np.load("/content/drive/MyDrive/final_slices_64-brain.npz")

In [ ]:
list(load)

['train', 'label']

In [ ]:
train1 = load['train']
label1 = load['label']

In [ ]:
#printing the shape of resized array
print(train1.shape)
print(label1.shape)

(18240, 64, 64)
(18240,)


In [ ]:
train1 = np.array(train1)
label1 = np.array(label1)

In [ ]:
train1

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [ ]:
label1

array(['Autism', 'Autism', 'Autism', ..., 'control', 'control', 'control'], dtype='<U7')

In [ ]:
le = preprocessing.LabelEncoder()

In [ ]:
#accessing gpu
tf.debugging.set_log_device_placement(True)

In [ ]:
Kfold=StratifiedKFold(n_splits=10 ,shuffle=True)
CVScores=[]
for train,test in Kfold.split(train1,label1):
  x_train=train1[train]
  x_test=train1[test]
  y_train=label1[train]
  y_test=label1[test]
  
  Y_train = le.fit_transform(y_train)
  Y_test = le.fit_transform(y_test)

  #model Architecture
  model=models.Sequential()
  model.add(layers.Conv2D(32,kernel_size=(3,3),input_shape=(64,64,1)))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.BatchNormalization())

 
  model.add(layers.Conv2D(64,kernel_size=(3,3)))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.BatchNormalization())
  model.add(layers.SpatialDropout2D(0.25))
 
  model.add(layers.Conv2D(128,kernel_size=(3,3),kernel_initializer='he_normal'))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.BatchNormalization())
  model.add(layers.SpatialDropout2D(0.25))
 
  model.add(layers.Conv2D(256,kernel_size=(3,3),kernel_initializer='he_normal'))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.BatchNormalization())
  model.add(layers.SpatialDropout2D(0.25))
 
  model.add(layers.Flatten())
  model.add(layers.Dense(256))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.25))

  model.add(layers.Dense(128))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.25))

  model.add(layers.Dense(64))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.25))
  model.add(layers.Dense(1, activation='sigmoid'))
  opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
  model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])
  with tf.device('/GPU:0'):  
   history=model.fit(x_train,Y_train,epochs=50,batch_size=128,validation_split=0.3)  
  scores=model.evaluate(x_test,Y_test,verbose=0)
  print("%s: %.2f%%"%(model.metrics_names[1],scores[1]*100))
  CVScores.append(scores[1]*100)
print("%.2f%%(+/-%.2f%%)"%(np.mean(CVScores),np.std(CVScores)))

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job

KeyboardInterrupt: ignored

In [ ]:
print(CVScores)

[74.39693212509155, 76.5350878238678, 74.45175647735596, 79.16666865348816, 77.08333134651184, 75.43859481811523, 75.43859481811523, 74.34210777282715, 76.5350878238678, 75.87719559669495]


In [ ]:
Kfold=StratifiedKFold(n_splits=10 ,shuffle=True)
CV_pre=[]
for train,test in Kfold.split(train1,label1):
  x_train=train1[train]
  x_test=train1[test]
  y_train=label1[train]
  y_test=label1[test]
  
  Y_train = le.fit_transform(y_train)
  Y_test = le.fit_transform(y_test)

  #model Architecture
  model=models.Sequential()
  model.add(layers.Conv2D(32,kernel_size=(3,3),input_shape=(64,64,1)))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.BatchNormalization())

 
  model.add(layers.Conv2D(64,kernel_size=(3,3)))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.BatchNormalization())
  model.add(layers.SpatialDropout2D(0.25))
 
  model.add(layers.Conv2D(128,kernel_size=(3,3),kernel_initializer='he_normal'))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.BatchNormalization())
  model.add(layers.SpatialDropout2D(0.25))
 
  model.add(layers.Conv2D(256,kernel_size=(3,3),kernel_initializer='he_normal'))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.BatchNormalization())
  model.add(layers.SpatialDropout2D(0.25))
 
  model.add(layers.Flatten())
  model.add(layers.Dense(256))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.25))

  model.add(layers.Dense(128))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.25))

  model.add(layers.Dense(64))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.25))
  model.add(layers.Dense(1, activation='sigmoid'))
  opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
  model.compile(loss='binary_crossentropy',optimizer=opt,metrics=[tf.keras.metrics.Precision()])
  with tf.device('/GPU:0'):  
   history=model.fit(x_train,Y_train,epochs=50,batch_size=128,validation_split=0.3)  
  scores=model.evaluate(x_test,Y_test,verbose=0)
  print("%s: %.2f%%"%(model.metrics_names[1],scores[1]*100))
  CV_pre.append(scores[1]*100)
print("%.2f%%(+/-%.2f%%)"%(np.mean(CV_pre),np.std(CV_pre)))

Streaming output truncated to the last 5000 lines.
51/90 [================>.............] - ETA: 1s - loss: 0.2894 - precision_9: 0.7948Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_582292 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_582292 in device /job:localhost/replica:0/task:0/device:GPU:0
53/90 [================>.............] - ETA: 1s - loss: 0.2892 - precision_9: 0.7941Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_582292 in device /job:localhost/replica:0/task:0/device

In [ ]:
print(CV_pre)

[92.64150857925415, 91.82608723640442, 94.84536051750183, 93.01886558532715, 93.06358098983765, 93.33333373069763, 96.26718759536743, 94.55782175064087, 95.9692895412445, 94.39071416854858]


In [ ]:
Kfold=StratifiedKFold(n_splits=10 ,shuffle=True)
CV_recall=[]
for train,test in Kfold.split(train1,label1):
  x_train=train1[train]
  x_test=train1[test]
  y_train=label1[train]
  y_test=label1[test]
  
  Y_train = le.fit_transform(y_train)
  Y_test = le.fit_transform(y_test)

  #model Architecture
  model=models.Sequential()
  model.add(layers.Conv2D(32,kernel_size=(3,3),input_shape=(64,64,1)))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.BatchNormalization())

 
  model.add(layers.Conv2D(64,kernel_size=(3,3)))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.BatchNormalization())
  model.add(layers.SpatialDropout2D(0.25))
 
  model.add(layers.Conv2D(128,kernel_size=(3,3),kernel_initializer='he_normal'))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.BatchNormalization())
  model.add(layers.SpatialDropout2D(0.25))
 
  model.add(layers.Conv2D(256,kernel_size=(3,3),kernel_initializer='he_normal'))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.BatchNormalization())
  model.add(layers.SpatialDropout2D(0.25))
 
  model.add(layers.Flatten())
  model.add(layers.Dense(256))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.25))

  model.add(layers.Dense(128))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.25))

  model.add(layers.Dense(64))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.25))
  model.add(layers.Dense(1, activation='sigmoid'))
  opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
  model.compile(loss='binary_crossentropy',optimizer=opt,metrics=[tf.keras.metrics.Recall()])
  with tf.device('/GPU:0'):  
   history=model.fit(x_train,Y_train,epochs=50,batch_size=128,validation_split=0.3)  
  scores=model.evaluate(x_test,Y_test,verbose=0)
  print("%s: %.2f%%"%(model.metrics_names[1],scores[1]*100))
  CV_recall.append(scores[1]*100)
print("%.2f%%(+/-%.2f%%)"%(np.mean(CV_recall),np.std(CV_recall)))


Streaming output truncated to the last 5000 lines.
51/90 [================>.............] - ETA: 1s - loss: 0.3192 - recall_9: 0.7256Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_889052 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_889052 in device /job:localhost/replica:0/task:0/device:GPU:0
53/90 [================>.............] - ETA: 1s - loss: 0.3206 - recall_9: 0.7242Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_889052 in device /job:localhost/replica:0/task:0/device:GPU:0

In [ ]:
print(CV_recall)

[60.41666865348816, 55.48245906829834, 58.004385232925415, 54.27631735801697, 57.7850878238678, 47.91666567325592, 47.14912176132202, 54.27631735801697, 50.65789222717285, 49.2324560880661]


In [ ]:
Kfold=StratifiedKFold(n_splits=10 ,shuffle=True)
CV_sensitivity=[]
for train,test in Kfold.split(train1,label1):
  x_train=train1[train]
  x_test=train1[test]
  y_train=label1[train]
  y_test=label1[test]
  
  Y_train = le.fit_transform(y_train)
  Y_test = le.fit_transform(y_test)

  #model Architecture
  model=models.Sequential()
  model.add(layers.Conv2D(32,kernel_size=(3,3),input_shape=(64,64,1)))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.BatchNormalization())

 
  model.add(layers.Conv2D(64,kernel_size=(3,3)))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.BatchNormalization())
  model.add(layers.SpatialDropout2D(0.25))
 
  model.add(layers.Conv2D(128,kernel_size=(3,3),kernel_initializer='he_normal'))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.BatchNormalization())
  model.add(layers.SpatialDropout2D(0.25))
 
  model.add(layers.Conv2D(256,kernel_size=(3,3),kernel_initializer='he_normal'))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.BatchNormalization())
  model.add(layers.SpatialDropout2D(0.25))
 
  model.add(layers.Flatten())
  model.add(layers.Dense(256))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.25))

  model.add(layers.Dense(128))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.25))

  model.add(layers.Dense(64))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.25))
  model.add(layers.Dense(1, activation='sigmoid'))
  opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
  model.compile(loss='binary_crossentropy',optimizer=opt,metrics=[tf.keras.metrics.SensitivityAtSpecificity(0.5)])
  with tf.device('/GPU:0'):  
   history=model.fit(x_train,Y_train,epochs=50,batch_size=128,validation_split=0.3)  
  scores=model.evaluate(x_test,Y_test,verbose=0)
  print("%s: %.2f%%"%(model.metrics_names[1],scores[1]*100))
  CV_sensitivity.append(scores[1]*100)
print("%.2f%%(+/-%.2f%%)"%(np.mean(CV_sensitivity),np.std(CV_sensitivity)))

NameError: ignored

In [ ]:
print(CV_sensitivity)

In [ ]:
Kfold=StratifiedKFold(n_splits=10 ,shuffle=True)
CV_specifity=[]
for train,test in Kfold.split(train1,label1):
  x_train=train1[train]
  x_test=train1[test]
  y_train=label1[train]
  y_test=label1[test]
  
  Y_train = le.fit_transform(y_train)
  Y_test = le.fit_transform(y_test)

  #model Architecture
  model=models.Sequential()
  model.add(layers.Conv2D(32,kernel_size=(3,3),input_shape=(64,64,1)))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.BatchNormalization())

 
  model.add(layers.Conv2D(64,kernel_size=(3,3)))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.BatchNormalization())
  model.add(layers.SpatialDropout2D(0.25))
 
  model.add(layers.Conv2D(128,kernel_size=(3,3),kernel_initializer='he_normal'))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.BatchNormalization())
  model.add(layers.SpatialDropout2D(0.25))
 
  model.add(layers.Conv2D(256,kernel_size=(3,3),kernel_initializer='he_normal'))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.BatchNormalization())
  model.add(layers.SpatialDropout2D(0.25))
 
  model.add(layers.Flatten())
  model.add(layers.Dense(256))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.25))

  model.add(layers.Dense(128))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.25))

  model.add(layers.Dense(64))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.25))
  model.add(layers.Dense(1, activation='sigmoid'))
  opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
  model.compile(loss='binary_crossentropy',optimizer=opt,metrics=[tf.keras.metrics.SpecificityAtSensitivity()])
  with tf.device('/GPU:0'):  
   history=model.fit(x_train,Y_train,epochs=50,batch_size=128,validation_split=0.3)  
  scores=model.evaluate(x_test,Y_test,verbose=0)
  print("%s: %.2f%%"%(model.metrics_names[1],scores[1]*100))
  CV_specifity.append(scores[1]*100)
print("%.2f%%(+/-%.2f%%)"%(np.mean(CV_specifity),np.std(CV_specifity)))

In [ ]:
print(CV_specifity)

In [ ]:
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 28.9 MB/s 


In [ ]:
import tensorflow_addons as tfa

In [ ]:
Kfold=StratifiedKFold(n_splits=10 ,shuffle=True)
CV_f1score=[]
for train,test in Kfold.split(train1,label1):
  x_train=train1[train]
  x_test=train1[test]
  y_train=label1[train]
  y_test=label1[test]
  
  Y_train = le.fit_transform(y_train)
  Y_test = le.fit_transform(y_test)

  #model Architecture
  model=models.Sequential()
  model.add(layers.Conv2D(32,kernel_size=(3,3),input_shape=(64,64,1)))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.BatchNormalization())

 
  model.add(layers.Conv2D(64,kernel_size=(3,3)))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.BatchNormalization())
  model.add(layers.SpatialDropout2D(0.25))
 
  model.add(layers.Conv2D(128,kernel_size=(3,3),kernel_initializer='he_normal'))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.BatchNormalization())
  model.add(layers.SpatialDropout2D(0.25))
 
  model.add(layers.Conv2D(256,kernel_size=(3,3),kernel_initializer='he_normal'))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.MaxPooling2D(pool_size=(2,2)))
  model.add(layers.BatchNormalization())
  model.add(layers.SpatialDropout2D(0.25))
 
  model.add(layers.Flatten())
  model.add(layers.Dense(256))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.25))

  model.add(layers.Dense(128))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.25))

  model.add(layers.Dense(64))
  model.add(layers.LeakyReLU(0.3))
  model.add(layers.BatchNormalization())
  model.add(layers.Dropout(0.25))
  model.add(layers.Dense(1, activation='sigmoid'))
  opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
  model.compile(loss='binary_crossentropy',optimizer=opt, metrics=[tf.keras.metrics.Accuracy(),
                           tfa.metrics.F1Score(num_classes=1, average='macro')])
  with tf.device('/GPU:0'):  
   history=model.fit(x_train,Y_train,epochs=50,batch_size=128,validation_split=0.3)  
  scores=model.evaluate(x_test,Y_test,verbose=0)
  print("%s: %.2f%%"%(model.metrics_names[1],scores[1]*100))
  CV_f1score.append(scores[1]*100)
print("%.2f%%(+/-%.2f%%)"%(np.mean(CV_f1score),np.std(CV_f1score)))

Streaming output truncated to the last 5000 lines.
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_5201 in device /job:localhost/replica:0/task:0/device:GPU:0
33/90 [==========>...................] - ETA: 1s - loss: 0.2306 - accuracy: 0.0000e+00 - f1_score: 0.4436Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_5201 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_5201 in device /job:localhost/replica:0/task:0/device:GPU:0
35/90 [==========>...................] - ETA: 1s - loss: 0.2296 - accuracy: 0.0000e+00 - f1_score: 0.4439Executing op ReadVariableOp in device /job

In [ ]:
print(CV_f1score)